<table align="left">
  <tr>
    <th colspan=2>Thông tin cá nhân</th>
  </tr>
  <tr>
    <td>Họ tên</td>
    <td>Kiều Công Hậu</td>
  </tr>
  <tr>
    <td>MSSV</td>
    <td><strong>18127259</strong></td>
  </tr>
  <tr>
    <td>Lớp</td>
    <td>18CLC1</td>
  </tr>
  <tr>
    <td>Môn học</td>
    <td>Toán Ứng dụng và Thống kê</td>
  </tr>
</table>

<h1 align="center">CÂN BẰNG PHẢN ỨNG HÓA HỌC</h1>

In [1]:
from sympy import *
from IPython.display import Latex

In [2]:
# Trả về chuỗi biểu diễn Latex của công thức hóa học mol
def chemical_formula(mol):
    formula = r""
    for e in mol:
        if isinstance(e[0], list):
            formula += "(" + chemical_formula(e[0]) + ")_" + str(e[1]) if e[1] != 1 else chemical_formula(e[0])
        else:
            formula += r"\text{" + e[0] + "}" + ("_" + str(e[1]) if e[1] != 1 else "")
    return formula

In [3]:
# Trả về tập hợp các nguyên tố phân biệt có trong 1 chất (mol)
def distinct_elements(mol):
    element_set = set()
    for e in mol:
        if isinstance(e[0], list):
            element_set.update(distinct_elements(e[0]))
        elif isinstance(e[0], str):
            element_set.add(e[0])
    return element_set

In [4]:
# Trả về số lượng nguyên tố (element) có trong một chất (mol)
def count(mol, element):
    for e in mol:
        if isinstance(e[0], list):
            return count(e[0], element) * e[1]
        elif isinstance(e[0], str) and e[0] == element:
            return e[1]
    return 0

# Trả về một ma trận A với kích thước mxn được thiết lập dựa vào danh sách các chất có trong PTHH (lhs và rhs)
def init_matrix(lhs, rhs):
    element_set = set()
    [element_set.update(distinct_elements(mol)) for mol in lhs]
    
    m, n = len(element_set), len(lhs) + len(rhs)
    matrix = [[0 for _ in range(n)] for _ in range(m)]
    
    for (element, mi) in zip(element_set, range(m)):
        for ni in range(len(lhs)):
            matrix[mi][ni] += count(lhs[ni], element)
        for ni in range(len(rhs)):
            matrix[mi][len(lhs) + ni] -= count(rhs[ni], element)
        
    return matrix

In [5]:
# Trả về danh sách các hệ số sau khi giải hệ PTTT được biểu diễn bằng ma trận matrix
def solve(matrix):
    n = len(matrix[0])
    coefficient_list = [1 for _ in range(n)]

    if Matrix(matrix).rank() != n - 1:
        temp = list(linsolve((Matrix(matrix), Matrix([0 for _ in range(len(matrix))])), [symbols(f"x{i + 1}") for i in range(n)]))
        coefficient_list = [temp[0][i] for i in range(n)]
    else:
        matrix = Matrix(matrix).rref()[0]
        for i in range(n - 1):
            coefficient_list[i] = -matrix[n*(i + 1) - 1]
    
    common_denominator = lcm([fraction(coefficient)[1] for coefficient in coefficient_list])
    for i in range(n):
        coefficient_list[i] *= common_denominator
    
    return coefficient_list

In [6]:
# Trả về chuỗi biểu diễn Latex của một chất (mol) kèm theo hệ số (coefficient) tương ứng
def equation_component(coefficient, mol, symbol=""):
    return (str(coefficient) if coefficient != 1 else "") + chemical_formula(mol) + symbol

# Trả về chuỗi biểu diễn Latex của PTHH đã được cân bằng
# dựa vào danh sách các chất trong PTHH và danh sách các hệ số tương ứng
def build_equation(coefficient_list, lhs, rhs):
    equation = r""
    
    for i in range(len(lhs) - 1):
        equation += equation_component(coefficient_list[i], lhs[i], " + ")
    equation += equation_component(coefficient_list[len(lhs) - 1], lhs[-1], r" \to ")
    
    for i in range(len(rhs) - 1):
        equation += equation_component(coefficient_list[len(lhs) + i], rhs[i], " + ")
    equation += equation_component(coefficient_list[-1], rhs[-1])
    
    return equation

In [7]:
# Trả về chuỗi biểu diễn Latex của PTHH đã được cân bằng
# dựa vào danh sách các chất trong PTHH
def balance_chemical_equation(lhs, rhs):
    matrix = init_matrix(lhs, rhs)
    coefficient_list = solve(matrix)
    return build_equation(coefficient_list, lhs, rhs)

In [8]:
Al_OH3 = [("Al", 1), ([("O", 1), ("H", 1)], 3)]
H2_SO4 = [("H", 2), ([("S", 1), ("O", 4)], 1)]
Al2_SO43 = [("Al", 2), ([("S", 1), ("O", 4)], 3)]
H2O = [("H", 2), ("O", 1)]

equation = balance_chemical_equation([Al_OH3, H2_SO4], [Al2_SO43, H2O])
Latex("$$" + equation + "$$")

<IPython.core.display.Latex object>

In [9]:
H2O = [("H", 2), ("O", 1)]
H2 = [("H", 2)]
O2 = [("O", 2)]

equation = balance_chemical_equation([H2O], [H2, O2])
Latex("$$" + equation + "$$")

<IPython.core.display.Latex object>

In [10]:
Al = [("Al", 1)]
HNO3 = [("H", 1), ("N", 1), ("O", 3)]
AlNO33 = [("Al", 1), ([("N", 1), ("O", 3)], 3)]
N2O = [("N", 2), ("O", 1)]
H2O = [("H", 2), ("O", 1)]

equation = balance_chemical_equation([Al, HNO3], [AlNO33, N2O, H2O])
Latex("$$" + equation + "$$")

<IPython.core.display.Latex object>